<a href="https://colab.research.google.com/github/Zaxx48/Mask_RCNN/blob/master/Park_01_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir("/content/drive/My Drive/AK49/Mask_RCNN/")

%tensorflow_version 1.x
import os
import numpy as np
import cv2
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
from pathlib import Path
import tensorflow.keras.preprocessing.image as imgpros
from google.colab.patches import cv2_imshow

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6


# Filter a list of Mask R-CNN detection results to get only the detected cars / trucks
def get_car_boxes(boxes, class_ids):
    car_boxes = []
    for i, box in enumerate(boxes):
        # If the detected object isn't a car / truck, skip it
        if class_ids[i] in [3, 8, 6]:
            car_boxes.append(box)
    return np.array(car_boxes)


# Root directory of the project
ROOT_DIR = Path("/content/drive/MyDrive/AK49/Mask_RCNN")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)
# Create a Mask-RCNN model in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

# Load pre-trained model
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:
# Location of parking spaces
file = open("/content/drive/MyDrive/AK49/Mask_RCNN/car_boxes/file_mov31.txt", "r")
#cambiar file por un archivo de texto con coordenadas
parkinglot_boxes = file.read()
parkarr = parkinglot_boxes.split('\n')
parkarr.pop(0)
print(parkarr)
pinned_car_boxes = []
for box in parkarr:
    box = box.split(' ')
    box[0] = int(box[0])
    box[1] = int(box[1])
    box[2] = int(box[2])
    box[3] = int(box[3])
    pinned_car_boxes.append([box[1], box[0], box[3], box[2]])
pinned_car_boxes = np.array(pinned_car_boxes)
print(pinned_car_boxes)

['14 791 266 881', '302 788 539 878', '617 797 809 899', '884 791 1079 893', '1139 779 1343 878', '1397 794 1598 875', '1646 782 1841 863', '5 656 224 728', '245 656 440 743', '473 668 644 749', '698 665 881 746', '932 677 1112 758', '1139 671 1316 752', '1384 683 1556 758', '1619 692 1793 776', '1739 656 1913 758', '21 504 91 517', '117 511 186 529', '234 514 306 546', '360 523 481 541', '544 519 655 543', '700 522 786 552', '846 525 946 558', '997 528 1111 561', '1167 528 1270 562', '1317 529 1432 564', '1468 532 1587 571', '1614 529 1708 592', '1800 528 1884 582']
[[ 791   14  881  266]
 [ 788  302  878  539]
 [ 797  617  899  809]
 [ 791  884  893 1079]
 [ 779 1139  878 1343]
 [ 794 1397  875 1598]
 [ 782 1646  863 1841]
 [ 656    5  728  224]
 [ 656  245  743  440]
 [ 668  473  749  644]
 [ 665  698  746  881]
 [ 677  932  758 1112]
 [ 671 1139  752 1316]
 [ 683 1384  758 1556]
 [ 692 1619  776 1793]
 [ 656 1739  758 1913]
 [ 504   21  517   91]
 [ 511  117  529  186]
 [ 514  234 

In [ ]:
# Video file or camera to process - set this to 0 to use your webcam instead of a video file
VIDEO_SOURCE = os.path.join(ROOT_DIR,"/test_videos/DJI_0031.MOV")

# Load the video file we want to run detection on
video_capture = cv2.VideoCapture(VIDEO_SOURCE)

# FPS calculate and timing to frames
seconds = 3
fps = video_capture.get(cv2.CAP_PROP_FPS)  # Gets the frames per second
multiplier = int(fps * seconds)

while video_capture.isOpened():
    success, frame = video_capture.read()
    if success:
        frameId = int(round(video_capture.get(1)))
        cv2.imwrite('test_images/new'+ str(frameId)+'.jpg', frame)
        if frameId % multiplier == 0:
            #rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_image = imgpros.img_to_array(frame)
            # Run the image through the Mask R-CNN model to get results.
            
            results = model.detect([rgb_image], verbose=0)
           
            # Mask R-CNN assumes we are running detection on multiple images.
            # We only passed in one image to detect, so only grab the first result.
            r = results[0]
            parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])

            overlaps = mrcnn.utils.compute_overlaps(pinned_car_boxes, parked_car_boxes)
            # overlaps = overlaps.transpose()

            free_space = False
            # for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):
            for parking_area, overlap_areas in zip(pinned_car_boxes, overlaps):
                max_IoU_overlap = np.max(overlap_areas)
                # x1, y1 ,x2, y2 = parking_area
                y1, x1, y2, x2 = parking_area
                if max_IoU_overlap < 0.25:
                    # Parking space not occupied! Draw a green box around it
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    # Flag that we have seen at least one open space
                    free_space = True
                else:
                    # Parking space is still occupied - draw a red box around it
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)

                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.3, (255, 255, 255))
            # for box in parked_car_boxes:
            #     print("Car: ", box)
            #
            #     y1, x1, y2, x2 = box
            #
            #     # Draw the box
            #     cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
            cv2_imshow(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else: break
video_capture.release()
cv2.destroyAllWindows()